In [1]:
# !pip install xlrd

In [1]:
!pip install openensembles

     |████████████████████████████████| 53 kB 638 kB/s 
     |████████████████████████████████| 5.2 MB 11.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 14.8 MB 33.8 MB/s 
     |████████████████████████████████| 22.3 MB 1.3 MB/s 
     |████████████████████████████████| 1.6 MB 39.5 MB/s 
     |████████████████████████████████| 11.5 MB 25.3 MB/s 
     |████████████████████████████████| 9.9 MB 25.5 MB/s 
     |████████████████████████████████| 27.4 MB 142 kB/s 
     |████████████████████████████████| 6.4 MB 32.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.27-cp37-cp37m-linux_x86_64.whl size=2311903 sha256=aa7d66b34367a1c031bc511ab20ab7fc8d4b770c36f4c391a47d1c810f79307f
  Stored in directory: /root/.cache/pip/wheels/73/5f/2f/9a25

In [3]:
# 第 1 部分
# 載入函式庫
import matplotlib.pyplot as plt
import numpy as np
import openensembles as oe 
import pandas as pd
from sklearn import metrics
# from sklearn.metrics.cluster import silhouette_score  # 輪廓係數 (樣本集,聚類標籤)


# 載入資料集
# data = pd.read_excel('/content/WHR.xls')
# regs = pd.read_excel('/content/REG.xls')

regs = pd.read_excel("/content/REG.xlsx")
data = pd.read_excel('/content/WHR.xlsx')

# 使用 2017 年的資料並填補缺少的項目
recents = data[data.Year == 2017]
recents = recents.dropna(axis=1, how="all") # axis=1: 刪除包含缺失值的列; how="all": 所有的值都缺失，才刪除行或列
recents = recents.fillna(recents.median())

# 使用以下特徵
columns = ['Log GDP per capita',
           'Social support',
           'Healthy life expectancy at birth',
           'Freedom to make life choices',
           'Generosity',
           'Perceptions of corruption',
           'Positive affect',
           'Negative affect',
           'Confidence in national government',
           'Democratic Quality',
           'Delivery Quality']

# 建立 data 物件
cluster_data = oe.data(recents[columns], columns)

In [13]:
# regs.head()

,Country name,Region
0,Finland,Western Europe
1,Denmark,Western Europe
2,Switzerland,Western Europe
3,Iceland,Western Europe
4,Netherlands,Western Europe


In [14]:
# data.head()

,Country name,Year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,...,GINI index (World Bank estimate),"GINI index (World Bank estimate), average 2000-16","gini of household income reported in Gallup, by wp5-year","Most people can be trusted, Gallup","Most people can be trusted, WVS round 1981-1984","Most people can be trusted, WVS round 1989-1993","Most people can be trusted, WVS round 1994-1998","Most people can be trusted, WVS round 1999-2004","Most people can be trusted, WVS round 2005-2009","Most people can be trusted, WVS round 2010-2014"
0,Afghanistan,2008,3.723590,7.168690,0.450662,50.799999,0.718114,0.177889,0.881686,0.517637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,2009,4.401778,7.333790,0.552308,51.200001,0.678896,0.200178,0.850035,0.583926,...,NaN,NaN,0.441906,0.286315,NaN,NaN,NaN,NaN,NaN,NaN
2,Afghanistan,2010,4.758381,7.386629,0.539075,51.599998,0.600127,0.134353,0.706766,0.618265,...,NaN,NaN,0.327318,0.275833,NaN,NaN,NaN,NaN,NaN,NaN
3,Afghanistan,2011,3.831719,7.415019,0.521104,51.919998,0.495901,0.172137,0.731109,0.611387,...,NaN,NaN,0.336764,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afghanistan,2012,3.782938,7.517126,0.520637,52.240002,0.530935,0.244273,0.775620,0.710385,...,NaN,NaN,0.344540,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# recents.median()

Year                                                        2017.000000
Life Ladder                                                    5.552915
Log GDP per capita                                             9.492126
Social support                                                 0.828646
Healthy life expectancy at birth                              66.000000
Freedom to make life choices                                   0.812530
Generosity                                                    -0.030673
Perceptions of corruption                                      0.782116
Positive affect                                                0.720716
Negative affect                                                0.279024
Confidence in national government                              0.481682
Democratic Quality                                            -0.164411
Delivery Quality                                              -0.173838
Standard deviation of ladder by country-year                   2

In [22]:
# 測試不同的子群數
Ks = [2, 4, 6, 8, 10, 12, 14]
# 測試不同的基學習器個數
sizes = [5, 10, 20, 50]
for K in Ks:
    for ensemble_size in sizes:
      print('子群數'+str(K),'基學習器個數'+str(ensemble_size))

子群數2 基學習器個數5
子群數2 基學習器個數10
子群數2 基學習器個數20
子群數2 基學習器個數50
子群數4 基學習器個數5
子群數4 基學習器個數10
子群數4 基學習器個數20
子群數4 基學習器個數50
子群數6 基學習器個數5
子群數6 基學習器個數10
子群數6 基學習器個數20
子群數6 基學習器個數50
子群數8 基學習器個數5
子群數8 基學習器個數10
子群數8 基學習器個數20
子群數8 基學習器個數50
子群數10 基學習器個數5
子群數10 基學習器個數10
子群數10 基學習器個數20
子群數10 基學習器個數50
子群數12 基學習器個數5
子群數12 基學習器個數10
子群數12 基學習器個數20
子群數12 基學習器個數50
子群數14 基學習器個數5
子群數14 基學習器個數10
子群數14 基學習器個數20
子群數14 基學習器個數50


In [23]:
# 使用openensembles共線鏈(Co-occurrence Linkage)集成非監督式學習的基學習器
# 共線矩陣顯示任何2筆資料被基學習器分在相同子群的頻率
# 共線鏈將共線矩陣視為相似度矩陣，接著使用階層式分群

In [36]:
# recents[columns].tail()

,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect,Confidence in national government,Democratic Quality,Delivery Quality
1653,9.439296,0.895879,66.300003,0.635505,-0.206035,0.843969,0.725643,0.362985,0.241124,-1.195530,-1.744199
1666,8.727759,0.828646,67.699997,0.812530,-0.030673,0.782116,0.720716,0.279024,0.481682,-0.548213,-0.227277
1677,9.492126,0.789555,55.900002,0.595191,-0.030673,0.782116,0.455182,0.295064,0.247787,-2.326562,-1.678789
1689,8.213179,0.743754,54.799999,0.823169,0.128904,0.739541,0.684623,0.387189,0.717004,-0.114061,-0.488385
1702,7.549491,0.754147,55.000000,0.752826,-0.069670,0.751208,0.806428,0.224051,0.682647,-0.988153,-1.350867


In [35]:
# preds.labels

{'co_occ_linkage': array([ 4, 11, 10, 11, 10, 14, 14, 10, 11,  8, 10, 14,  3,  8, 11,  7, 10,
         9,  4,  2,  4, 14,  1,  1, 12, 11, 11,  6,  4, 12, 12, 14, 12, 14,
        10, 11,  2, 10, 12,  5, 14, 14,  5,  3,  8, 14,  6, 14,  8,  4,  3,
        11, 13,  9, 14,  5,  2, 10,  5, 14, 14, 14,  1, 11, 13, 10,  8,  5,
         8,  9,  8,  5,  9, 10,  1,  3,  2,  9, 14, 11,  5,  6,  4, 14,  6,
         9, 11,  8,  2, 11, 10,  4,  5,  6,  2, 14, 14, 11,  4,  1, 14,  5,
        10, 12, 10, 11,  2, 12, 14,  9,  8,  5, 10,  5, 11,  1, 13, 12, 12,
         6, 14,  4, 14, 11, 14, 14,  9,  8,  6, 11,  3,  8, 10, 10,  2,  3,
         8,  9, 14, 12, 12,  8, 10, 11,  6,  3,  3], dtype=int32)}

In [25]:
# 第 2 部分程式
# 建立集成模型，測試2,4,6,8,10,12,14不同的子群數，同時也測試5,10,20,50個基學習器
np.random.seed(123456)
results = {'K':[], 'size':[], 'silhouette': []}
# 測試不同的子群數
Ks = [2, 4, 6, 8, 10, 12, 14]
# 測試不同的基學習器個數
sizes = [5, 10, 20, 50]
for K in Ks:
    for ensemble_size in sizes:
        # 初始化基學習器
        ensemble = oe.cluster(cluster_data)
        for i in range(ensemble_size):
            # 訓練基學習器
            name = f'kmeans_{ensemble_size}_{i}'
            ensemble.cluster('parent', 'kmeans', name, K)

        # 使用共現鏈組成所有基學習器的輸出
        preds = ensemble.finish_co_occ_linkage(threshold = 0.5)
        print(f'K: {K}, size {ensemble_size}:', end=' ')
        # 計算輪廓係數
        silhouette = metrics.silhouette_score(recents[columns],preds.labels['co_occ_linkage']) #輪廓係數  (X, labels)(樣本集,聚類標籤)
        print('%.2f' % silhouette)
        results['K'].append(K)
        results['size'].append(ensemble_size)
        results['silhouette'].append(silhouette)
        
results_df = pd.DataFrame(results)
cross = pd.crosstab(results_df.K,
                    results_df['size'], 
                    results_df['silhouette'],
                    aggfunc=lambda x: x)
print(cross)

K: 2, size 5: 0.62
K: 2, size 10: 0.62
K: 2, size 20: 0.62
K: 2, size 50: 0.62
K: 4, size 5: 0.53
K: 4, size 10: 0.53
K: 4, size 20: 0.53
K: 4, size 50: 0.53
K: 6, size 5: 0.48
K: 6, size 10: 0.48
K: 6, size 20: 0.48
K: 6, size 50: 0.48
K: 8, size 5: 0.34
K: 8, size 10: 0.40
K: 8, size 20: 0.34
K: 8, size 50: 0.34
K: 10, size 5: 0.29
K: 10, size 10: 0.36
K: 10, size 20: 0.32
K: 10, size 50: 0.29
K: 12, size 5: 0.30
K: 12, size 10: 0.29
K: 12, size 20: 0.35
K: 12, size 50: 0.35
K: 14, size 5: 0.31
K: 14, size 10: 0.32
K: 14, size 20: 0.31
K: 14, size 50: 0.34
size        5         10        20        50
K                                           
2     0.617703  0.617703  0.617703  0.617703
4     0.533317  0.533317  0.533317  0.533317
6     0.475362  0.475362  0.475362  0.475362
8     0.343622  0.396019  0.343622  0.343622
10    0.294892  0.355284  0.321448  0.285979
12    0.304994  0.291727  0.349968  0.349781
14    0.309338  0.322273  0.312236  0.343352


In [27]:
results_df.tail()

,K,size,silhouette
23,12,50,0.349781
24,14,5,0.309338
25,14,10,0.322273
26,14,20,0.312236
27,14,50,0.343352


In [32]:
pd.crosstab(results_df['K'],
                    results_df['size'], 
                    results_df['silhouette'],
                    aggfunc=lambda x: x)

size,5,10,20,50
K,,,,
2,0.617703,0.617703,0.617703,0.617703
4,0.533317,0.533317,0.533317,0.533317
6,0.475362,0.475362,0.475362,0.475362
8,0.343622,0.396019,0.343622,0.343622
10,0.294892,0.355284,0.321448,0.285979
12,0.304994,0.291727,0.349968,0.349781
14,0.309338,0.322273,0.312236,0.343352
